# Kubernetes

**Sources**

- [Hello minikube](https://kubernetes.io/docs/tutorials/hello-minikube/) tutorial on official k8s site.
- [kubernetes crashcourse](https://www.youtube.com/watch?v=s_o8dwzRlu4) video on youtube.

## Architecture

K8s cluster generally consits of:

- **Control plane nodes**: the brain of the cluster, contains **control plane agents**:

    - API server (kube-api): Provides an RESTful interface to manage the cluster.
    - The scheduler (kube-scheduler): assigns new workloads, such as pods to the nodes, which are typicaly workers.
    - Controller Managers. A watch-loop application that periodically checks the state of the cluster. There are two types of the controller managers: kube-controller-manager and cloud-controller-manager.
    - Key-Value Data Store ([etcd](https://etcd.io/)): distributed key-value map.
    - Node Agent - kubelet.
    - Proxy - kube-proxy.

- **Worker nodes**: machines on which pods are executed. A worker consists of the following components:

    - Container runtime: The tool to run containers. It could be Docker Engine, CRI-O, containerd, and Mirantis.
    - Node Agent - kubelet.
    - Proxy - kube-proxy.
    - Optional add-ons: dashborads, cluster-level monitoring, and logging.


A **pod** is the smallest unit in the Kubernetes. It literally represents an application. In the container runtime, it looks like a group of the containers that are deployed to handle a single application.

**Proxy** is a special kube-proxy application installed on each node, that manages the node's the network.

### Kubelet

A Kubelet is a special process in each node, that receives pod information from the API server and runs it in the container runtime.

To communicate with a particular container runtime kubelete uses special abstraction service: "CRI shim". CRI: container runtime interface.

The Kubelet uses a special abstraction service to communicate with a specific container runtime. This abstraction service hides the implementation details of the specific service.

![](kubernetes_files/Kubelet.svg)

The following CRI shims exist: cri-containerd, CRI-O, and cri-dockerd. Their names make it clear to which container runtimes they correspond. 

## Setup

For local experimentation setup you can try following solutions:

- [Minikube](https://minikube.sigs.k8s.io/docs/) single- and multi-node local K8s cluster, recommended for a learning environment deployed on a single host.
- [Kind](https://kind.sigs.k8s.io/) Multi-node Kubernetes cluster deployed in Docker containers acting as Kubernetes nodes, recommended for a learning environment.
- [Microk8s](https://microk8s.io/) Local and cloud K8s cluster for developers and production, from Canonical.
- [K3S](https://k3s.io/) Lightweight Kubernetes cluster for local, cloud, edge, IoT deployments, originally from Rancher, currently a CNCF project.

### Minikube

Minikube is local Kubernetes, focusing on making it easy to learn and develop for Kubernetes. Read more in [start with minikube](https://minikube.sigs.k8s.io/docs/start/?arch=%2Flinux%2Fx86-64%2Fstable%2Fbinary+download) page. This is a tool that we'll use to play with kubernetes.

Check more examples in the [Minikube](kubernetes/minikube.ipynb) page.

---

Start the Minikube cluster using the `minikube start` command.

In [23]:
minikube start &> /dev/null

Use the command `minikube status` to check the status of your cluster.

In [24]:
minikube status

minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured



Use `minikube delete` to remove the current cluster of the minikube.

In [25]:
minikube delete

🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /home/user/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


#### Autocompletion

To set up `TAB` completion, add the script printed to the standard output by `minikube completion bash` to the source: `source <(minikube completion bash)`. To enable the completion when running the shell, add the command to the `~/.bashrc` file: `echo 'source <(minikube completion bash)' >> ~/.bashrc`.

## kubectl

`kubectl` is a command line interface that allows you to interact with Kubernetes.

Check [installation guide](https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/) for `kubectl`.

For me, using the ubuntu option `snap install kubectl --classic` worked well.

To enable `TAB` completion add `source` the script printed with `kubectl completion bash`: `source <(kubectl completion bash)`. To enable completion at the start of the shell, add line into `~/.bashrc` file: `echo 'source <(kubectl completion bash)' >> ~/.bashrc`.

If you're using `minikube`, you can access the `kubectl` using the command `minikube kubectl -- <subcommands>`. This will give you access to `kubectl` for minikube.

**Note:** Even for learning purposes, we'll use a separate installation of the `kubectl`, as this installation closer to the real-world challanges.

---

The following cell shows the version of the cluster displayed using `minikube kubectl -- version --client`.

In [7]:
minikube kubectl -- version --client

Client Version: v1.34.0
Kustomize Version: v5.7.1


If you have successfully installed `kubectl` you will be able to run the following cell:

In [2]:
kubectl version

Client Version: v1.34.1
Kustomize Version: v5.7.1
Server Version: v1.34.0


### Configuration

The behavior of the `kubectl` is typically defined by the configuration file that is located in the `~/.kube/config` directory.

---

The following cell displays a few lines of the `kubectl` configuration.

In [2]:
cat ~/.kube/config | head -n 10

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /home/user/.minikube/ca.crt
    extensions:
    - extension:
        last-update: Mon, 13 Oct 2025 09:07:40 CEST
        provider: minikube.sigs.k8s.io
        version: v1.37.0
      name: cluster_info


### Cluster information

The `kubectl cluster-info` command displays the information about the cluster that `kubectl` currently connected to.

In [3]:
kubectl cluster-info

Kubernetes control plane is running at https://192.168.49.2:8443
CoreDNS is running at https://192.168.49.2:8443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


### Proxy

Since `kubectl` is just convenient wrapper around the Kubernetes API, it can communicate with the kubernetes API and allow you to create a locally hosted proxy for it. Use the `kubectl proxy` command for this. 

---

The following cell runs the proxy.

In [14]:
kubectl proxy &
export proxy_pid=$!

[1] 101050


Now you are able to access api:

In [15]:
curl localhost:8001/api

Starting to serve on 127.0.0.1:8001
{
  "kind": "APIVersions",
  "versions": [
    "v1"
  ],
  "serverAddressByClientCIDRs": [
    {
      "clientCIDR": "0.0.0.0/0",
      "serverAddress": "192.168.49.2:8443"
    }
  ]
}


In [16]:
kill $proxy_pid

## Pods

In k8s, a pod is an abstraction representing an application. In the container runtime, it is ususally represented by a set of the containers/volumes in the container runtime.

The are following operations that you can do with the pods:

| Opearation | `kubectl` command      |
|------------|------------------------|
| Create     | `kubectl create`       |
| Run        | `kubectl run`          |
| Get pods   | `kubectl get pods`     |
| Describe   | `kubectl describe pod` |

---

The following cell uses the command `kubectl run` to create and run the pod.

In [38]:
kubectl run nginx-pod --image=nginx:1.22.1

pod/nginx-pod created


The corresponding pod appears in the output of the `kubectl get pods` command.

In [33]:
kubectl get pods

NAME                       READY   STATUS             RESTARTS       AGE
bla-bla-7c4989cf78-9gmxz   0/1     CrashLoopBackOff   9 (3m7s ago)   24m
nginx-pod                  1/1     Running            0              2s


Use the command `kubectl delete pod <pod name>` to delete the pod. The following cell deletes the pod that was created earlier and lists the pods.

In [39]:
kubectl delete pod nginx-pod
kubectl get pods

pod "nginx-pod" deleted from default namespace
NAME                       READY   STATUS             RESTARTS         AGE
bla-bla-7c4989cf78-9gmxz   0/1     CrashLoopBackOff   10 (2m25s ago)   28m


## Core components

Here we will discuss components of the kubernetes:

 - **Pod** is a group of containers.
 - **Deployment** is an abstraction that manages pods.
 - **Service** is pod that was exposed.

Creating `hello-node` deployment.

In [ ]:
!kubectl create deployment hello-node --image=registry.k8s.io/e2e-test-images/agnhost:2.39 -- /agnhost netexec --http-port=8080

deployment.apps/hello-node created


Now we can access the deployments list:

In [ ]:
!kubectl get deployments

NAME         READY   UP-TO-DATE   AVAILABLE   AGE
hello-node   1/1     1            1           51s


It have a pod that can be listed using the appropriate command:

In [ ]:
!kubectl get pods

NAME                          READY   STATUS    RESTARTS   AGE
hello-node-55fdcd95bf-v9w97   1/1     Running   0          2m18s


Now we only have a basic service that runs on k8s.

In [ ]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   5m17s


But after exposing `hello-node` we'll have one more service.

In [ ]:
%%bash
kubectl expose deployment hello-node --type=LoadBalancer --port=8080
kubectl get services

Error from server (AlreadyExists): services "hello-node" already exists


NAME         TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
hello-node   LoadBalancer   10.97.143.213   <pending>     8080:32657/TCP   90s
kubernetes   ClusterIP      10.96.0.1       <none>        443/TCP          7m57s
